In [2]:
import os
import sys

#collection = os.environ.get('COLLECTION', 'new_youtube_collection').strip()

In [3]:
import uuid
session_id = uuid.uuid4()
session_id = str(session_id)
session_id

'd9732969-fd14-4c6f-b043-9a8c967096b9'

In [4]:
os.environ['USER'] = 'postgres'
user = os.environ.get('USER', 'postgres').strip()

postgres


In [2]:
#collection

In [3]:
import os
import logging
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.vectorstores.pgvector import PGVector
from langchain_community.document_loaders import UnstructuredExcelLoader
#from langchain_openai import OpenAIEmbeddings

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.getenv("MISTRAL_API_KEY")

In [5]:
model_name="mistralai/Mistral-7B-Instruct-v0.1"

In [6]:
#_, file_extension = os.path.splitext()
file_path="/Users/mohitverma/Documents/Etech-RAG-Chatbot/Etech-RAG-E2E/qevalprofaq 5.csv"
loader = CSVLoader(file_path)

In [7]:
print(loader.load()[0].page_content)

breadcrumb_title: Can a user view my message if I am replying to some other user in chat feature?
post_content: Except for an agent, every user role will have two chat modes on their consoles. These include a Private mode and a general reply mode. A user can use a private mode tab to establish a private conversation for evaluation with other user, thus shielding off other users to view their messages. Similarly, a user can use a reply mode to send the messages, which could be viewed by all the other users.


In [8]:
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_community.llms import HuggingFaceEndpoint
from dotenv import load_dotenv

load_dotenv()

embed_model = HuggingFaceEmbedding("sentence-transformers/all-mpnet-base-v2")

/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
model=HuggingFaceEndpoint(repo_id=model_name, HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

WARNING! HUGGINGFACEHUB_API_TOKEN is not default parameter.
                    HUGGINGFACEHUB_API_TOKEN was transferred to model_kwargs.
                    Please make sure that HUGGINGFACEHUB_API_TOKEN is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/mohitverma/.cache/huggingface/token
Login successful


In [12]:

repo_id="mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(repo_id=repo_id,
                          max_length=128,
                          temperature=0.7)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/mohitverma/.cache/huggingface/token
Login successful


In [13]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
import os
from unittest import loader

import pandas as pd
import sys
from dotenv import load_dotenv
from logger import logging

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.vectorstores.pgvector import PGVector
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from db_connector import connection_string

from constants import *

load_dotenv()

#embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

class DataIngestionProcessing:
    
    def __init__(self, file_path):
        self.file_path = file_path

    def split_data(self):
        # Determine the file type based on the extension
        _ ,file_extension = os.path.splitext(self.file_path)

        if file_extension.lower() == '.csv':
            logging.info(f"Reading CSV file : {self.file_path}")
            print("Reading CSV file")
            loader = CSVLoader(self.file_path)

        elif file_extension.lower() == '.xlsx':
            logging.info(f"Reading Excel file : {self.file_path}")
            loader=UnstructuredExcelLoader(self.file_path, mode="elements")

        elif file_extension.lower() == '.pdf':
            logging.info(f"Reading PDF file {self.file_path}")
            loader=PyPDFLoader(self.file_path)

        else:
            raise ValueError(f"File extension {file_extension} not supported")


        data = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=150)
        docs = text_splitter.split_documents(data)
        return docs


    def push_data(self, docs):
        #PGVector.delete_collection(PGSQL_COLLECTION)
        print(f"{self.file_path} is Pushed successfully into {PGSQL_COLLECTION}")
        PGVector.from_documents(
            documents=docs,
            embedding=embeddings,
            collection_name=PGSQL_COLLECTION,
            connection_string= connection_string,
            pre_delete_collection=False
            )
        print(f"{self.file_path} is Pushed successfully into {PGSQL_COLLECTION}")
        
    
doc_processor = DataIngestionProcessing(file_path="/Users/mohitverma/Documents/Etech-RAG-Chatbot/Etech-RAG-E2E/qevalprofaq 5.csv")
docs = doc_processor.split_data()
doc_processor.push_data(docs)


Reading CSV file
/Users/mohitverma/Documents/Etech-RAG-Chatbot/Etech-RAG-E2E/qevalprofaq 5.csv is Pushed successfully into etech_collection


/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/langchain_community/vectorstores/pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


/Users/mohitverma/Documents/Etech-RAG-Chatbot/Etech-RAG-E2E/qevalprofaq 5.csv is Pushed successfully into etech_collection


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

In [ ]:
def delete_collection(self) -> None:
    self.logger.debug("Trying to delete collection")
    with Session(self._conn) as session:
        collection = self.get_collection(session)
        if not collection:
            self.logger.warning("Collection not found")
            return
        session.delete(collection)
        session.commit()

In [16]:
from langchain_community.document_loaders import TextLoader
#from langchain_core.language_models.chat_models import LangSmithParams
#from langchain_mistralai.chat_models import ChatMistralAI
from langchain_ai21 import ChatAI21
from langchain_ai21 import AI21Embeddings
#from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
import os

load_dotenv()


# Load data
loader = CSVLoader("/Users/mohitverma/Documents/Etech-RAG-Chatbot/Etech-RAG-E2E/qevalprofaq 5.csv")
docs = loader.load()
# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Define the embedding model
embeddings = AI21Embeddings()
# Create the vector store 
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
model = ChatAI21(temperature=0.0, max_tokens=4096)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
print(response["answer"])

KVlTLRfckUbPN81UiTC6OPtO1tjzLbC6


NameError: name 'CSVLoader' is not defined

In [22]:

from dotenv import load_dotenv
import os

load_dotenv()

embeddings = AI21Embeddings(api_key= os.getenv('AI21_API_KEY'))
llm=ChatAI21(model='jamba-instruct',temperature=0.0, max_tokens=4096,api_key=os.getenv('AI21_API_KEY'))

KVlTLRfckUbPN81UiTC6OPtO1tjzLbC6


In [5]:
from colorama import Fore, Style
import pyfiglet

# Initialize colorama
from colorama import init
init()

# Fancy title
title = pyfiglet.figlet_format("Fancy Input")
print(Fore.CYAN + title + Style.RESET_ALL)

# Custom input prompt
user_name = input(Fore.GREEN + "Enter your name: " + Style.RESET_ALL)
age = input(Fore.YELLOW + "Enter your age: " + Style.RESET_ALL)

print(Fore.MAGENTA + f"Hello, {user_name}! You are {age} years old." + Style.RESET_ALL)


 _____                        ___                   _   
|  ___|_ _ _ __   ___ _   _  |_ _|_ __  _ __  _   _| |_ 
| |_ / _` | '_ \ / __| | | |  | || '_ \| '_ \| | | | __|
|  _| (_| | | | | (__| |_| |  | || | | | |_) | |_| | |_ 
|_|  \__,_|_| |_|\___|\__, | |___|_| |_| .__/ \__,_|\__|
                      |___/            |_|              

Hello, sx! You are c years old.
